# 1. Carga de archivos

In [ ]:
# Librerías
from pathlib import Path
import pandas as pd


In [ ]:
# Funciones
%run 00_functions.ipynb

In [ ]:
# Crea una lista de los nombres de los archivos que existen en la carpeta data
ruta_archivos = Path("../data/raw")
archivos = [archivo for archivo in ruta_archivos.iterdir()]

In [ ]:
# Se valida las columnas de los archivos
validador_de_columnas(archivos)

In [ ]:
# Leer y concatena todos los archivos
columnas_seleccionadas = pd.read_csv(archivos[0])
columnas_seleccionadas = columnas_seleccionadas.columns # Se seleccionan las columnas

df = [pd.read_csv(archivo, header= None, skiprows=1) for archivo in archivos] # Se leen los archivos omitiendo la fila 1 y se indica que no hay encabezados
df = pd.concat(df, ignore_index=True) # Concatena los dataframe
df.columns = columnas_seleccionadas # Se agregan los encabezados
print(df.shape)
print(df.head())